## Create Plots of Channel Conveyance Change

This notebook creates the plots of channel conveyance change that pop up as a 'tool tip' when a user scrolls over a gauge site using the results data.

Import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import datetime as dt
import matplotlib as mpl
from datetime import timedelta
from scipy import stats
# sns.set()

fttom = 0.3048

Import gauge results

In [2]:
gage_info = pd.read_csv(r'study_data\gauge_results.csv')

Define function to compute linear trend to fit channel change for sites that can be described by a linear analysis

In [3]:
def trend_conf_int(x, y):
    x = x
    y = y
    lsq_res = stats.linregress(x, y)
    pred_y = lsq_res[1] + lsq_res[0] * x
    pred_x = np.linspace(np.min(x),np.max(x), len(x))
    return(pred_x, pred_y, lsq_res[0], lsq_res[2]*lsq_res[2], lsq_res[3])

Define an offset date to compute change with respect to:

In [4]:
offset_date = pd.Timestamp('2015-01-01')

Define a function that will compute closest time to offset date:

In [5]:
def nearest(items, pivot):
    return min(items, key=lambda x: abs(x - pivot))

Load data, make & save plots

In [8]:
for i, gage in gage_info.iterrows():
    gage_id = gage['#']
    gagename = gage['USGS Gagename']
    gageno = gage['USGS Gageno']
    channel_data_fn = r'study_data\channel_conveyance_change\%s_%s_channel_capacity_change.csv'%(gagename, gageno)
    channel_data = pd.read_csv(channel_data_fn)
    
    channel_data['DateTime'] = pd.to_datetime(channel_data.DateTime)

    channel_data['median']= channel_data['q_resid_cms'].rolling(5, center=True).median(centered=True)

    channel_data = channel_data.set_index('DateTime', drop=True)
    #drop duplicate indicies
    channel_data = channel_data.loc[~channel_data.index.duplicated(), :]
    #offset to zero at 2015
    closest_time = nearest(channel_data.index, offset_date)
    offset_channel = channel_data['median'].loc[closest_time]
    channel_data.q_resid_cms -= offset_channel
    channel_data['median'] -= offset_channel

    f, ax = plt.subplots(figsize=(6,2))
    ax.scatter(
        channel_data.index,
        channel_data['q_resid_cms'],
        c='tomato',
        marker='o',
        s=10
    )
    if gage.category in ['minimal', 'monotonic']:
        pred_x, channel_data['pred_fit'], slope, r2, pval =  trend_conf_int(
            channel_data.index.map(dt.datetime.toordinal),
            channel_data.q_resid_cms
        )
        ax.plot(
            channel_data.index,
            channel_data.pred_fit,
            c='tomato',
            linestyle='--',
            linewidth=2
        )
    else:
        ax.plot(
            channel_data.index,
            channel_data['median'],
            c='tomato',
            linestyle='--',
            linewidth=2
        )
    ax.set_ylabel('conveyance \n change '+ r'$(m^3/s)$', fontsize=13)
    ax.set_xlabel('date', fontsize=14)
    ax.set_title('%d: %s (%s)'%(gage_id, gagename, gageno), fontsize=16)
    ax.set_xlim(pd.to_datetime('01-01-1940'), pd.to_datetime('01-01-2020'))
    plt.xticks(rotation=40, fontsize=12)
    plt.yticks(fontsize=12)
    plt.savefig(r'figures\channel_conveyance_change\%s_%s.png'%(gagename, gageno),
                dpi=300,
                bbox_inches='tight')
    plt.close()

C:\Users\sahrendt\Anaconda3\envs\uwgda2020\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\sahrendt\Anaconda3\envs\uwgda2020\lib\site-packages\scipy\stats\_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\sahrendt\Anaconda3\envs\uwgda2020\lib\site-packages\scipy\stats\_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


ConversionError: Failed to convert value(s) to axis units: masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True],
       fill_value=1e+20,
            dtype=float64)

ConversionError: Failed to convert value(s) to axis units: masked_array(data=[--, --, --, --, --, --, --, --, --, --, --, --, --, --],
             mask=[ True,  True,  True,  True,  True,  True,  True,  True,
                    True,  True,  True,  True,  True,  True],
       fill_value=1e+20,
            dtype=float64)

<Figure size 432x144 with 1 Axes>